In [1]:
!pip install torch torchvision torchaudio
!pip install ogb
!pip install torch-geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu1

In [ ]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

wandb.login(key="Your-api-key")
Dataset_name = "IMDB-BINARY"

# Learnable Root-Based BFS Positional Encoding GCN

In [3]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch import nn
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_undirected, degree
import wandb
from collections import deque

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = f"GCN Graph classification({Dataset_name})_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def prep_graph(d):
    d = d.clone()
    if d.edge_index.numel() > 0:
        d.edge_index = to_undirected(d.edge_index, num_nodes=d.num_nodes)
    if d.x is None:
        d.x = torch.ones((d.num_nodes, 1), dtype=torch.float)
    deg = degree(d.edge_index[0], d.num_nodes, dtype=torch.float) if d.edge_index.numel() > 0 else torch.zeros(d.num_nodes, dtype=torch.float)
    m = max(deg.max().item(), 1.0)
    deg = (deg / m).unsqueeze(1)
    d.x = torch.cat([d.x, deg], dim=1)
    return d

def bfs_rank_from_root(num_nodes, edge_index_local, root_local):
    adj = [[] for _ in range(num_nodes)]
    u, v = edge_index_local
    for i in range(u.numel()):
        a = int(u[i])
        b = int(v[i])
        if a != b:
            adj[a].append(b)
            adj[b].append(a)
    ranks = torch.full((num_nodes,), -1.0)
    if num_nodes == 0 or root_local < 0 or root_local >= num_nodes:
        return ranks
    q = deque([root_local])
    seen = [False] * num_nodes
    seen[root_local] = True
    order = []
    while q:
        cur = q.popleft()
        order.append(cur)
        for w in adj[cur]:
            if not seen[w]:
                seen[w] = True
                q.append(w)
    L = max(1, len(order))
    for i, node in enumerate(order):
        ranks[node] = float(i) / float(L)
    return ranks

class LearnableBFS_GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, bfs_dim=8, tau=1.0):
        super().__init__()
        self.tau = tau
        self.root_scorer = nn.Sequential(
            nn.Linear(in_channels, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
        self.bfs_proj = nn.Sequential(
            nn.Linear(1, 16),
            nn.ReLU(),
            nn.Linear(16, bfs_dim)
        )
        self.conv1 = GCNConv(in_channels + bfs_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, out_channels)

    @torch.no_grad()
    def _compute_bfs_rank_batch(self, x, edge_index, batch, root_logits):
        device = x.device
        N = x.size(0)
        ranks = torch.full((N,), -1.0, device='cpu')
        edge_index_cpu = edge_index.detach().cpu()
        batch_cpu = batch.detach().cpu()
        logits_cpu = root_logits.detach().cpu().squeeze(-1)
        num_graphs = int(batch_cpu.max().item()) + 1 if N > 0 else 0
        for g in range(num_graphs):
            idx = (batch_cpu == g).nonzero(as_tuple=False).view(-1)
            n = idx.numel()
            if n == 0:
                continue
            logits_g = logits_cpu[idx]
            probs_g = F.gumbel_softmax(logits_g, tau=self.tau, hard=True)
            root_local = int(probs_g.argmax().item())
            map_gl2lc = torch.full((N,), -1, dtype=torch.long)
            map_gl2lc[idx] = torch.arange(n, dtype=torch.long)
            u_all, v_all = edge_index_cpu
            m_u = map_gl2lc[u_all]
            m_v = map_gl2lc[v_all]
            mask = (m_u >= 0) & (m_v >= 0)
            u_loc = m_u[mask]
            v_loc = m_v[mask]
            ranks_g = bfs_rank_from_root(n, (u_loc, v_loc), root_local)
            ranks[idx] = ranks_g
        return ranks.to(device)

    def forward(self, x, edge_index, batch):
        root_logits = self.root_scorer(x)
        bfs_rank = self._compute_bfs_rank_batch(x, edge_index, batch, root_logits)
        bfs_feat = self.bfs_proj(bfs_rank.unsqueeze(1))
        x = torch.cat([x, bfs_feat], dim=1)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def run_seed(seed):
    set_seed(seed)
    dataset = TUDataset(root='data/TUDataset', name=Dataset_name, transform=prep_graph)
    if dataset.num_features == 0:
        for data in dataset:
            data.x = torch.ones((data.num_nodes, 1))
    dataset = dataset.shuffle()
    num_graphs = len(dataset)
    train_size = int(0.8 * num_graphs)
    train_dataset = dataset[:train_size]
    test_dataset = dataset[train_size:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    in_channels = dataset[0].x.size(1)
    model = LearnableBFS_GCN(in_channels, 64, dataset.num_classes, bfs_dim=8, tau=1.0).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_LearnableBFS_STG_seed_{seed}",
        group=f"GCN_{Dataset_name}_LearnableBFS",
        job_type="learnable_bfs",
        reinit=True,
        config={
            "model": "GCN",
            "hidden_channels": 64,
            "epochs": EPOCHS,
            "lr": 0.001,
            "dataset": Dataset_name,
            "task": "Learnable BFS",
            "positional_encoding": "Learned root (ST-Gumbel) + BFS rank MLP",
            "seed": seed
        }
    )

    def train():
        model.train()
        total_loss = 0.0
        correct = 0
        total_samples = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            total_samples += data.num_graphs
        train_acc = correct / total_samples
        return total_loss / len(train_loader), train_acc

    def test(loader):
        model.eval()
        correct = 0
        total_samples = 0
        for data in loader:
            data = data.to(device)
            out = model(data.x, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            total_samples += data.num_graphs
        return correct / total_samples

    for epoch in range(1, EPOCHS + 1):
        loss, train_acc = train()
        test_acc = test(test_loader)
        print(f"Seed {seed} | Epoch {epoch:03d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})

    wandb.finish()

if __name__ == "__main__":
    wandb.login()
    for s in range(1, NUM_SEEDS + 1):
        run_seed(s)


Processing...
Done!
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250822_131848-sau9o16t
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run GCN_LearnableBFS_STG_seed_1
wandb: ⭐️ View project at https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: 🚀 View run at https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/sau9o16t


Seed 1 | Epoch 001 | Loss: 0.6904 | Train Acc: 0.5212 | Test Acc: 0.5000
Seed 1 | Epoch 002 | Loss: 0.6872 | Train Acc: 0.5525 | Test Acc: 0.6250
Seed 1 | Epoch 003 | Loss: 0.6852 | Train Acc: 0.5525 | Test Acc: 0.6100
Seed 1 | Epoch 004 | Loss: 0.6847 | Train Acc: 0.5637 | Test Acc: 0.6100
Seed 1 | Epoch 005 | Loss: 0.6870 | Train Acc: 0.5775 | Test Acc: 0.6000
Seed 1 | Epoch 006 | Loss: 0.6830 | Train Acc: 0.5725 | Test Acc: 0.6100
Seed 1 | Epoch 007 | Loss: 0.6839 | Train Acc: 0.5763 | Test Acc: 0.6200
Seed 1 | Epoch 008 | Loss: 0.6830 | Train Acc: 0.5687 | Test Acc: 0.6250
Seed 1 | Epoch 009 | Loss: 0.6838 | Train Acc: 0.5825 | Test Acc: 0.6250
Seed 1 | Epoch 010 | Loss: 0.6837 | Train Acc: 0.5700 | Test Acc: 0.6050
Seed 1 | Epoch 011 | Loss: 0.6842 | Train Acc: 0.5775 | Test Acc: 0.6200
Seed 1 | Epoch 012 | Loss: 0.6841 | Train Acc: 0.5800 | Test Acc: 0.6150
Seed 1 | Epoch 013 | Loss: 0.6832 | Train Acc: 0.5825 | Test Acc: 0.6150
Seed 1 | Epoch 014 | Loss: 0.6838 | Train Acc: 0.58

wandb: uploading output.log; uploading config.yaml
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇██
wandb:      loss █▅▄▃▅▃▂▃▃▃▃▂▃▃▂▃▂▄▂▂▃▅▂▂▂▂▂▃▂▂▁▁▁▂▁▁▁▂▁▁
wandb:  test_acc ▁█▇▇▆▇██▇▇▇▇█▇█▇██▇▇█▇▇█▇▇█▇█▇▇█▇█▇█▇▇▇▇
wandb: train_acc ▁▄▄▅▇▇▆▇▆▇▇▇▆▇▇▄▆▆▇▇▅▄▇██▇▇▇▇█▇▇▇▇███▇█▇
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.68199
wandb:  test_acc 0.62
wandb: train_acc 0.58125
wandb: 
wandb: 🚀 View run GCN_LearnableBFS_STG_seed_1 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/sau9o16t
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_131848-sau9o16t/logs
wandb: Tracking run with wandb versio

Seed 2 | Epoch 001 | Loss: 0.6960 | Train Acc: 0.4725 | Test Acc: 0.4700
Seed 2 | Epoch 002 | Loss: 0.6906 | Train Acc: 0.5537 | Test Acc: 0.5050
Seed 2 | Epoch 003 | Loss: 0.6875 | Train Acc: 0.5400 | Test Acc: 0.4950
Seed 2 | Epoch 004 | Loss: 0.6845 | Train Acc: 0.5587 | Test Acc: 0.5850
Seed 2 | Epoch 005 | Loss: 0.6823 | Train Acc: 0.5613 | Test Acc: 0.5000
Seed 2 | Epoch 006 | Loss: 0.6825 | Train Acc: 0.5800 | Test Acc: 0.5700
Seed 2 | Epoch 007 | Loss: 0.6855 | Train Acc: 0.5513 | Test Acc: 0.5050
Seed 2 | Epoch 008 | Loss: 0.6827 | Train Acc: 0.5663 | Test Acc: 0.5950
Seed 2 | Epoch 009 | Loss: 0.6826 | Train Acc: 0.5850 | Test Acc: 0.5700
Seed 2 | Epoch 010 | Loss: 0.6802 | Train Acc: 0.5925 | Test Acc: 0.5600
Seed 2 | Epoch 011 | Loss: 0.6808 | Train Acc: 0.5900 | Test Acc: 0.5750
Seed 2 | Epoch 012 | Loss: 0.6816 | Train Acc: 0.5925 | Test Acc: 0.5650
Seed 2 | Epoch 013 | Loss: 0.6836 | Train Acc: 0.5825 | Test Acc: 0.5700
Seed 2 | Epoch 014 | Loss: 0.6801 | Train Acc: 0.58

wandb: uploading wandb-summary.json; uploading config.yaml
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
wandb:      loss █▆▅▄▃▄▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂▂▂▁▂▂▁▂▃▂▁
wandb:  test_acc ▁▃▂▇▃▃█▇▆▇▇▇▆▇▇▆▇▇▇▇▇▆▇▇▆▆▇▅▇▆▂▇▇▆▇▆▇▇▆▇
wandb: train_acc ▁▅▅▆▆▅▇█▇█▇█▇█▇█▇██▇▇█▇▆█████▇▇▇█▇▇█▇▆▇▇
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.67766
wandb:  test_acc 0.575
wandb: train_acc 0.5875
wandb: 
wandb: 🚀 View run GCN_LearnableBFS_STG_seed_2 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/lz0w6ls9
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_132124-lz0w6ls9/logs
wandb: Tracking run with wand

...

wandb: uploading output.log; uploading config.yaml
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss █▆▆▄▄▃▃▃▃▃▂▂▂▃▄▃▃▃▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▁▁▁▁▂▁
wandb:  test_acc ▂▁▅▅▂▅▆▆█▅▅▅▅▆▅▆▆▆▅▆▇▆▆▅▅▅▆▅▅▆▆▅▅▆▅▅▅▆▆▅
wandb: train_acc ▁▃▅▄▅▆▆▇▆▇█▇▇█▇▇▇▇▇███▇██▇▇█▇▇▇▇▇▇▇▇███▇
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.67759
wandb:  test_acc 0.555
wandb: train_acc 0.59375
wandb: 
wandb: 🚀 View run GCN_LearnableBFS_STG_seed_28 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/8amhguz2
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_143020-8amhguz2/logs
wandb: Tracking run with wandb vers

Seed 29 | Epoch 001 | Loss: 0.6904 | Train Acc: 0.5500 | Test Acc: 0.4850
Seed 29 | Epoch 002 | Loss: 0.6890 | Train Acc: 0.5275 | Test Acc: 0.5650
Seed 29 | Epoch 003 | Loss: 0.6862 | Train Acc: 0.5525 | Test Acc: 0.5700
Seed 29 | Epoch 004 | Loss: 0.6863 | Train Acc: 0.5413 | Test Acc: 0.5450
Seed 29 | Epoch 005 | Loss: 0.6848 | Train Acc: 0.5775 | Test Acc: 0.5450
Seed 29 | Epoch 006 | Loss: 0.6816 | Train Acc: 0.5850 | Test Acc: 0.6050
Seed 29 | Epoch 007 | Loss: 0.6824 | Train Acc: 0.5900 | Test Acc: 0.5550
Seed 29 | Epoch 008 | Loss: 0.6822 | Train Acc: 0.5938 | Test Acc: 0.5500
Seed 29 | Epoch 009 | Loss: 0.6830 | Train Acc: 0.5825 | Test Acc: 0.5350
Seed 29 | Epoch 010 | Loss: 0.6821 | Train Acc: 0.5813 | Test Acc: 0.5700
Seed 29 | Epoch 011 | Loss: 0.6820 | Train Acc: 0.5887 | Test Acc: 0.5400
Seed 29 | Epoch 012 | Loss: 0.6814 | Train Acc: 0.5725 | Test Acc: 0.5500
Seed 29 | Epoch 013 | Loss: 0.6843 | Train Acc: 0.5587 | Test Acc: 0.5600
Seed 29 | Epoch 014 | Loss: 0.6807 | T

wandb: uploading output.log; uploading config.yaml
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
wandb:      loss █▇▅▅▄▃▃▃▃▃▄▂▃▄▄▂▃▃▁▂▂▁▂▂▂▁▁▂▂▂▁▂▁▁▂▁▂▁▁▃
wandb:  test_acc ▁▅▅▄▄▄▄▄▄▅▅█▄▇▄▅█▇▄▇▄█▇▇▄▇████▇███▇▇██▇▇
wandb: train_acc ▃▁▄▂▆██▇▇▆▆▆▅▄▅▆▆▆▇▇▇▇▆▇▆▇▇▆▇▇▇▇▇▇▆██▇▅▇
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.68055
wandb:  test_acc 0.61
wandb: train_acc 0.58875
wandb: 
wandb: 🚀 View run GCN_LearnableBFS_STG_seed_29 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/pq48dr1p
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_143303-pq48dr1p/logs
wandb: Tracking run with wandb versi

Seed 30 | Epoch 001 | Loss: 0.6994 | Train Acc: 0.4688 | Test Acc: 0.4350
Seed 30 | Epoch 002 | Loss: 0.6900 | Train Acc: 0.5175 | Test Acc: 0.5250
Seed 30 | Epoch 003 | Loss: 0.6852 | Train Acc: 0.5825 | Test Acc: 0.6100
Seed 30 | Epoch 004 | Loss: 0.6846 | Train Acc: 0.5763 | Test Acc: 0.6050
Seed 30 | Epoch 005 | Loss: 0.6849 | Train Acc: 0.5487 | Test Acc: 0.5900
Seed 30 | Epoch 006 | Loss: 0.6834 | Train Acc: 0.5775 | Test Acc: 0.5900
Seed 30 | Epoch 007 | Loss: 0.6817 | Train Acc: 0.5863 | Test Acc: 0.5900
Seed 30 | Epoch 008 | Loss: 0.6830 | Train Acc: 0.5775 | Test Acc: 0.5400
Seed 30 | Epoch 009 | Loss: 0.6836 | Train Acc: 0.5713 | Test Acc: 0.6050
Seed 30 | Epoch 010 | Loss: 0.6846 | Train Acc: 0.5437 | Test Acc: 0.6100
Seed 30 | Epoch 011 | Loss: 0.6822 | Train Acc: 0.5700 | Test Acc: 0.5900
Seed 30 | Epoch 012 | Loss: 0.6820 | Train Acc: 0.5625 | Test Acc: 0.5950
Seed 30 | Epoch 013 | Loss: 0.6805 | Train Acc: 0.5837 | Test Acc: 0.5900
Seed 30 | Epoch 014 | Loss: 0.6827 | T

wandb: uploading config.yaml
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss █▃▃▃▂▂▂▃▂▂▂▂▂▂▂▂▂▁▁▂▂▂▂▁▁▂▁▁▁▂▁▂▁▂▂▁▁▁▁▁
wandb:  test_acc ▁▅██▇▅██▇▇███▇█▇███▇██▇████▇▇▇██████▇█▇█
wandb: train_acc ▁▄▇▇▇▇▅▇▆▇▇▆▇▇█▇████▇██▇██▇▇█▇█▇▇▇▆████▇
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.67966
wandb:  test_acc 0.6
wandb: train_acc 0.58125
wandb: 
wandb: 🚀 View run GCN_LearnableBFS_STG_seed_30 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/dixnvtzs
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_143545-dixnvtzs/logs


# **Layerwise BFS**

In [4]:
# **Layerwise BFS for Protein**
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = f"GCN Graph classification({Dataset_name})_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, bfs_dim=8):
        super().__init__()
        self.bfs_proj = torch.nn.Sequential(
            torch.nn.Linear(1, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, bfs_dim)
        )
        self.conv1 = GCNConv(in_channels + bfs_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels + bfs_dim, hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, x, bfs_rank, edge_index, batch):
        bfs_feat = self.bfs_proj(bfs_rank.unsqueeze(1))
        x = torch.cat([x, bfs_feat], dim=1)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        bfs_feat2 = self.bfs_proj(bfs_rank.unsqueeze(1))
        x = torch.cat([x, bfs_feat2], dim=1)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def make_bfs_rank_transform():
    def transform(data):
        G = to_networkx(data, to_undirected=True)
        bfs_rank = torch.full((data.num_nodes,), -1.0, dtype=torch.float)
        if G.number_of_nodes() > 0:
            degrees = dict(G.degree())
            root = max(degrees, key=degrees.get) if degrees else 0
            bfs_order = list(nx.bfs_tree(G, root))
            L = max(1, len(bfs_order))
            for i, node in enumerate(bfs_order):
                bfs_rank[node] = i / L
        data.bfs_rank = bfs_rank
        # REDDIT-BINARY has no node features; use degree as feature or 1s
        if not hasattr(data, 'x') or data.x is None:
            deg = torch.tensor([d for _, d in G.degree()], dtype=torch.float).unsqueeze(1)
            data.x = deg
        return data
    return transform

def run_seed(SEED):
    set_seed(SEED)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    dataset = TUDataset(root='data/TUDataset', name=Dataset_name, transform=make_bfs_rank_transform())
    torch.manual_seed(SEED)
    dataset = dataset.shuffle()
    train_dataset = dataset[:len(dataset)//2]
    test_dataset = dataset[len(dataset)//2:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_Layerwise_BFS_{Dataset_name}_seed_{SEED}",
        group=f"GCN_{Dataset_name}_Layerwise_BFS",
        job_type="layerwise-bfs",
        reinit=True,
        config={
            "model": "GCN",
            "hidden_channels": 64,
            "epochs": EPOCHS,
            "lr": 0.001,
            "dataset": Dataset_name,
            "task": "Layerwise_BFS_Injection",
            "seed": SEED,
            "cuda_available": torch.cuda.is_available(),
            "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
            "cuda_version": torch.version.cuda if torch.cuda.is_available() else None
        }
    )

    model = GCN(
        in_channels=dataset.num_features,
        hidden_channels=64,
        out_channels=dataset.num_classes
    ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    def train():
        model.train()
        total_loss = 0.0
        correct = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.bfs_rank, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * data.num_graphs
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
        acc = correct / len(train_dataset)
        return total_loss / len(train_dataset), acc
    
    def test():
        model.eval()
        correct = 0
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.bfs_rank, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
        return correct / len(test_dataset)
    
    for epoch in range(1, EPOCHS + 1):
        loss, train_acc = train()
        test_acc = test()
        print(f"Seed {SEED} | Epoch {epoch:03d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})
    wandb.finish()

if __name__ == "__main__":
    wandb.login()
    for s in range(1, NUM_SEEDS + 1):
        run_seed(s)


wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250822_143826-f4osbm02
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run GCN_Layerwise_BFS_IMDB-BINARY_seed_1
wandb: ⭐️ View project at https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: 🚀 View run at https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/f4osbm02


Seed 1 | Epoch 001 | Loss: 0.7165 | Train Acc: 0.4760 | Test Acc: 0.5080
Seed 1 | Epoch 002 | Loss: 0.7010 | Train Acc: 0.5160 | Test Acc: 0.4920
Seed 1 | Epoch 003 | Loss: 0.6942 | Train Acc: 0.5120 | Test Acc: 0.4920
Seed 1 | Epoch 004 | Loss: 0.6963 | Train Acc: 0.4960 | Test Acc: 0.6560
Seed 1 | Epoch 005 | Loss: 0.7008 | Train Acc: 0.5040 | Test Acc: 0.5080
Seed 1 | Epoch 006 | Loss: 0.6958 | Train Acc: 0.5000 | Test Acc: 0.5080
Seed 1 | Epoch 007 | Loss: 0.7007 | Train Acc: 0.5020 | Test Acc: 0.4920
Seed 1 | Epoch 008 | Loss: 0.7060 | Train Acc: 0.4980 | Test Acc: 0.5140
Seed 1 | Epoch 009 | Loss: 0.6970 | Train Acc: 0.5500 | Test Acc: 0.5080
Seed 1 | Epoch 010 | Loss: 0.6980 | Train Acc: 0.4820 | Test Acc: 0.6720
Seed 1 | Epoch 011 | Loss: 0.6923 | Train Acc: 0.5820 | Test Acc: 0.5080
Seed 1 | Epoch 012 | Loss: 0.7056 | Train Acc: 0.5040 | Test Acc: 0.4920
Seed 1 | Epoch 013 | Loss: 0.6883 | Train Acc: 0.5480 | Test Acc: 0.5080
Seed 1 | Epoch 014 | Loss: 0.6988 | Train Acc: 0.53

wandb: uploading wandb-summary.json; uploading config.yaml
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss █▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▅▄▄▅▃▃▂▂▂▂▁▂▂▃▂▁▁
wandb:  test_acc ▁▁▆▁▁▂▁▆▁▁▁▄▆▃▅▄▆▆▇▆▆▆▁▄▅▄▇▆█▇▇██▇▇▇▇▆▇▇
wandb: train_acc ▁▂▂▂▂▂▄▁▅▂▃▃▅▃▃▄▆▆▅▆▆▇▆▅▅▆▆▄▆▇▇▇█▇█▇▆▇█▇
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.61211
wandb:  test_acc 0.706
wandb: train_acc 0.648
wandb: 
wandb: 🚀 View run GCN_Layerwise_BFS_IMDB-BINARY_seed_1 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/f4osbm02
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_143826-f4osbm02/logs
wandb: Tracking run w

Seed 2 | Epoch 001 | Loss: 0.7060 | Train Acc: 0.4700 | Test Acc: 0.4980
Seed 2 | Epoch 002 | Loss: 0.7073 | Train Acc: 0.5280 | Test Acc: 0.5940
Seed 2 | Epoch 003 | Loss: 0.6985 | Train Acc: 0.5020 | Test Acc: 0.6660
Seed 2 | Epoch 004 | Loss: 0.6908 | Train Acc: 0.5560 | Test Acc: 0.6260
Seed 2 | Epoch 005 | Loss: 0.6874 | Train Acc: 0.5960 | Test Acc: 0.5760
Seed 2 | Epoch 006 | Loss: 0.6849 | Train Acc: 0.5880 | Test Acc: 0.6520
Seed 2 | Epoch 007 | Loss: 0.6888 | Train Acc: 0.5860 | Test Acc: 0.4980
Seed 2 | Epoch 008 | Loss: 0.6831 | Train Acc: 0.5820 | Test Acc: 0.5460
Seed 2 | Epoch 009 | Loss: 0.6737 | Train Acc: 0.6360 | Test Acc: 0.6180
Seed 2 | Epoch 010 | Loss: 0.6714 | Train Acc: 0.6260 | Test Acc: 0.5520
Seed 2 | Epoch 011 | Loss: 0.6618 | Train Acc: 0.6660 | Test Acc: 0.6360
Seed 2 | Epoch 012 | Loss: 0.6634 | Train Acc: 0.6320 | Test Acc: 0.4980
Seed 2 | Epoch 013 | Loss: 0.6524 | Train Acc: 0.6240 | Test Acc: 0.4980
Seed 2 | Epoch 014 | Loss: 0.6420 | Train Acc: 0.64

wandb: uploading output.log; uploading config.yaml
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss ███▇▇▇▇▇▆▆▆▅▅▅▄▃▃▂▂▂▃▂▂▂▂▂▂▂▂▂▁▁▁▁▂▂▁▁▂▁
wandb:  test_acc ▁▄▆▅▄▁▃▅▃▅▁▆▇█▅▅▇▇▇▇▇▆▇▇▇█▇▇████████▇▇█▇
wandb: train_acc ▁▃▂▃▄▄▄▆▅▆▅▆▅▇▇▇███▇█▇███▇▇█████████████
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.55117
wandb:  test_acc 0.696
wandb: train_acc 0.718
wandb: 
wandb: 🚀 View run GCN_Layerwise_BFS_IMDB-BINARY_seed_2 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/b56xz1gv
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_143931-b56xz1gv/logs
wandb: Tracking run with wand

...

wandb: uploading output.log; uploading config.yaml
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss ▇▆▆██▆▆▅▅▅▅▆▄▄▄▄▄▄▂▃▂▃▃▂▂▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁
wandb:  test_acc ▁▆▅▂▅▆▄▆▅▆▁▆▆▆▅██▅▆▇████▇█████▇██▇▇▇██▇█
wandb: train_acc ▁▄▄▂▂▅▄▅▄▅▆▄▅▆▇▄▆▇█▆▇▇▇▇▇▇█▇▇▇█▇██▇▇██▇█
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.5789
wandb:  test_acc 0.708
wandb: train_acc 0.724
wandb: 
wandb: 🚀 View run GCN_Layerwise_BFS_IMDB-BINARY_seed_28 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/q9qaptes
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_150959-q9qaptes/logs
wandb: Tracking run with wand

Seed 29 | Epoch 001 | Loss: 0.7079 | Train Acc: 0.5300 | Test Acc: 0.5080
Seed 29 | Epoch 002 | Loss: 0.6988 | Train Acc: 0.5080 | Test Acc: 0.4920
Seed 29 | Epoch 003 | Loss: 0.6934 | Train Acc: 0.5700 | Test Acc: 0.6820
Seed 29 | Epoch 004 | Loss: 0.6899 | Train Acc: 0.5840 | Test Acc: 0.4920
Seed 29 | Epoch 005 | Loss: 0.6939 | Train Acc: 0.5340 | Test Acc: 0.6000
Seed 29 | Epoch 006 | Loss: 0.6882 | Train Acc: 0.5400 | Test Acc: 0.5820
Seed 29 | Epoch 007 | Loss: 0.6982 | Train Acc: 0.5480 | Test Acc: 0.4920
Seed 29 | Epoch 008 | Loss: 0.6912 | Train Acc: 0.5860 | Test Acc: 0.4920
Seed 29 | Epoch 009 | Loss: 0.6956 | Train Acc: 0.5540 | Test Acc: 0.5920
Seed 29 | Epoch 010 | Loss: 0.6852 | Train Acc: 0.5600 | Test Acc: 0.6600
Seed 29 | Epoch 011 | Loss: 0.6833 | Train Acc: 0.5520 | Test Acc: 0.5740
Seed 29 | Epoch 012 | Loss: 0.6981 | Train Acc: 0.5240 | Test Acc: 0.4920
Seed 29 | Epoch 013 | Loss: 0.6802 | Train Acc: 0.6000 | Test Acc: 0.6040
Seed 29 | Epoch 014 | Loss: 0.6705 | T

wandb: uploading wandb-summary.json; uploading config.yaml
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss ██▇▇▇▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▃▃▃▄▂▂▂▂▂▁▁▂▂▃▃▁▁▁▁▁
wandb:  test_acc ▂▁▇▁▄▁▁▄▆▄▄▆▁▆▇▇▇▆▆▇▇███▇▆▇███▇█▇▇▇█▇██▇
wandb: train_acc ▂▁▃▄▂▂▄▃▃▃▄▅▅▅▅▆▇▆▅▇█▇▇▇▇█▇▇██▇▇▆▇██▇███
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.58278
wandb:  test_acc 0.686
wandb: train_acc 0.698
wandb: 
wandb: 🚀 View run GCN_Layerwise_BFS_IMDB-BINARY_seed_29 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/o0ci7jvi
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_151106-o0ci7jvi/logs
wandb: Tracking run 

Seed 30 | Epoch 001 | Loss: 0.7050 | Train Acc: 0.5060 | Test Acc: 0.5080
Seed 30 | Epoch 002 | Loss: 0.6995 | Train Acc: 0.5000 | Test Acc: 0.4920
Seed 30 | Epoch 003 | Loss: 0.6904 | Train Acc: 0.5460 | Test Acc: 0.5740
Seed 30 | Epoch 004 | Loss: 0.6901 | Train Acc: 0.5860 | Test Acc: 0.4980
Seed 30 | Epoch 005 | Loss: 0.6901 | Train Acc: 0.5960 | Test Acc: 0.5500
Seed 30 | Epoch 006 | Loss: 0.6884 | Train Acc: 0.6240 | Test Acc: 0.6480
Seed 30 | Epoch 007 | Loss: 0.6848 | Train Acc: 0.6280 | Test Acc: 0.5760
Seed 30 | Epoch 008 | Loss: 0.6870 | Train Acc: 0.5960 | Test Acc: 0.5480
Seed 30 | Epoch 009 | Loss: 0.6801 | Train Acc: 0.6380 | Test Acc: 0.6040
Seed 30 | Epoch 010 | Loss: 0.6801 | Train Acc: 0.6060 | Test Acc: 0.6240
Seed 30 | Epoch 011 | Loss: 0.6827 | Train Acc: 0.6060 | Test Acc: 0.6020
Seed 30 | Epoch 012 | Loss: 0.6726 | Train Acc: 0.6320 | Test Acc: 0.6340
Seed 30 | Epoch 013 | Loss: 0.6704 | Train Acc: 0.5960 | Test Acc: 0.6040
Seed 30 | Epoch 014 | Loss: 0.6636 | T

wandb: uploading wandb-summary.json; uploading config.yaml
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
wandb:      loss ██▇▇▇▇▇▇▇▆▆▆▆▆▅▄▄▃▃▃▃▃▂▃▃▂▂▂▂▂▃▂▂▂▁▂▂▁▁▁
wandb:  test_acc ▁▁▃▁▆▃▄▅▄▅▅▄▃▆▇▅▇▇▇▇▇█▆▇▇▆▇██▇█▇██▇▇▇▇██
wandb: train_acc ▁▃▄▄▅▄▆▅▅▅▆▆▄▄▆▆▇█▇▇▇█▇▇▆▇▇█▇█▆▇█████▇██
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.57195
wandb:  test_acc 0.716
wandb: train_acc 0.7
wandb: 
wandb: 🚀 View run GCN_Layerwise_BFS_IMDB-BINARY_seed_30 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal/runs/y70pl47d
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28IMDB-BINARY%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_151212-y70pl47d/logs


# **GCN Learnable BFS Sinusoidal**

In [3]:
import os, random, numpy as np, torch, math
import torch.nn.functional as F
from torch import nn
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

Dataset_name = "IMDB-BINARY"
EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = f"GCN Graph classification({Dataset_name})_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class LearnableBFSPositionalEncoding(nn.Module):
    def __init__(self, dim, max_len=512):
        super().__init__()
        self.lin = nn.Linear(1, dim)
        pe = torch.zeros(max_len, dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, dim, 2).float() * (-math.log(10000.0) / dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe)
        self.max_len = max_len
    def forward(self, bfs_rank):
        idx = (bfs_rank.clamp(0, 1) * (self.max_len - 1)).long()
        return self.lin(bfs_rank.unsqueeze(1)) + self.pe[idx]

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, bfs_dim=32, max_len=512):
        super().__init__()
        self.bfs_enc = LearnableBFSPositionalEncoding(bfs_dim, max_len)
        self.conv1 = GCNConv(in_channels + bfs_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, out_channels)
    def forward(self, x, bfs_rank, edge_index, batch):
        if x is None:
            x = torch.ones((bfs_rank.size(0), 1), device=bfs_rank.device, dtype=torch.float)
        x = torch.cat([x, self.bfs_enc(bfs_rank)], dim=1)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def make_bfs_rank_transform():
    def transform(data):
        G = to_networkx(data, to_undirected=True)
        bfs_rank = torch.full((data.num_nodes,), 0.0, dtype=torch.float)
        if G.number_of_nodes() > 0:
            deg = dict(G.degree())
            root = max(deg, key=deg.get) if len(deg) > 0 else 0
            order = list(nx.bfs_tree(G, root))
            L = max(0, len(order) - 1)
            if L == 0:
                for n in order:
                    bfs_rank[n] = 0.0
            else:
                for i, node in enumerate(order):
                    bfs_rank[node] = i / L
        data.bfs_rank = bfs_rank
        if data.x is None:
            data.x = torch.ones((data.num_nodes, 1), dtype=torch.float)
        return data
    return transform

def run_seed(SEED):
    set_seed(SEED)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dataset = TUDataset(root="data/TUDataset", name=Dataset_name, transform=make_bfs_rank_transform())
    torch.manual_seed(SEED)
    dataset = dataset.shuffle()
    num_graphs = len(dataset)
    train_size = int(0.8 * num_graphs)
    train_dataset = dataset[:train_size]
    test_dataset = dataset[train_size:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_LearnableSinBFS_seed_{SEED}",
        group=f"GCN_{Dataset_name}_LearnableSinBFS",
        job_type="learnable-sin-bfs",
        reinit=True,
        config={
            "model": "GCN",
            "hidden_channels": 64,
            "epochs": EPOCHS,
            "lr": 0.001,
            "dataset": Dataset_name,
            "task": "Learnable_Sinusoidal_BFS",
            "seed": SEED,
            "cuda_available": torch.cuda.is_available(),
            "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
            "cuda_version": torch.version.cuda if torch.cuda.is_available() else None
        }
    )

    in_ch = max(1, dataset.num_features)
    model = GCN(
        in_channels=in_ch,
        hidden_channels=64,
        out_channels=dataset.num_classes,
        bfs_dim=32,
        max_len=512
    ).to(device)

    opt = torch.optim.Adam(model.parameters(), lr=0.001)

    def train():
        model.train()
        total_loss = 0.0
        correct = 0
        total = 0
        for data in train_loader:
            data = data.to(device)
            out = model(data.x, data.bfs_rank, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            opt.zero_grad()
            loss.backward()
            opt.step()
            total_loss += loss.item() * data.num_graphs
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            total += data.num_graphs
        return total_loss / len(train_dataset), correct / total

    def test():
        model.eval()
        correct = 0
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.bfs_rank, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
        return correct / len(test_dataset)

    for epoch in range(1, EPOCHS + 1):
        loss, train_acc = train()
        test_acc = test()
        print(f"Seed {SEED} | Epoch {epoch:03d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})

    wandb.finish()

if __name__ == "__main__":
    wandb.login()
    for s in range(1, NUM_SEEDS + 1):
        run_seed(s)


Processing...
Done!
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Seed 1 | Epoch 001 | Loss: 0.6957 | Train Acc: 0.5150 | Test Acc: 0.4750
Seed 1 | Epoch 002 | Loss: 0.6949 | Train Acc: 0.5025 | Test Acc: 0.5200
Seed 1 | Epoch 003 | Loss: 0.6895 | Train Acc: 0.5262 | Test Acc: 0.5200
Seed 1 | Epoch 004 | Loss: 0.6863 | Train Acc: 0.5375 | Test Acc: 0.5200
Seed 1 | Epoch 005 | Loss: 0.6817 | Train Acc: 0.5625 | Test Acc: 0.6150
Seed 1 | Epoch 006 | Loss: 0.6809 | Train Acc: 0.5713 | Test Acc: 0.5700
Seed 1 | Epoch 007 | Loss: 0.6852 | Train Acc: 0.5525 | Test Acc: 0.6200
Seed 1 | Epoch 008 | Loss: 0.6771 | Train Acc: 0.5975 | Test Acc: 0.6200
Seed 1 | Epoch 009 | Loss: 0.6770 | Train Acc: 0.5475 | Test Acc: 0.5850
Seed 1 | Epoch 010 | Loss: 0.6769 | Train Acc: 0.5713 | Test Acc: 0.6300
Seed 1 | Epoch 011 | Loss: 0.6727 | Train Acc: 0.5525 | Test Acc: 0.6200
Seed 1 | Epoch 012 | Loss: 0.6688 | Train Acc: 0.5887 | Test Acc: 0.5650
Seed 1 | Epoch 013 | Loss: 0.6763 | Train Acc: 0.5663 | Test Acc: 0.6100
Seed 1 | Epoch 014 | Loss: 0.6941 | Train Acc: 0.52

epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,███▇▇▇▇▇▇▇▇█▇▇▆▆▆▅▅▅▅▅▅▄▄▅▄▄▄▃▃▄▃▃▃▃▂▁▂▁
test_acc,▁▃▃▆▄▆▅▇▆▄▃▃▃▇▇▇▇▇▇▇▆▇▇▅▇█▄▅▇█▃▄▇▇▇▆█▇▅▅
train_acc,▁▁▂▂▃▃▄▃▃▃▃▂▄▅▅▆▆▆▆▆▆▆▅▆▅▅▆▆▆▇▅▇▆▇▇▇▇█▇▇
epoch,50
loss,0.58386
test_acc,0.58
train_acc,0.67375


Seed 2 | Epoch 001 | Loss: 0.6985 | Train Acc: 0.4988 | Test Acc: 0.5200
Seed 2 | Epoch 002 | Loss: 0.6923 | Train Acc: 0.5200 | Test Acc: 0.5700
Seed 2 | Epoch 003 | Loss: 0.6931 | Train Acc: 0.5188 | Test Acc: 0.5150
Seed 2 | Epoch 004 | Loss: 0.6846 | Train Acc: 0.5550 | Test Acc: 0.4850
Seed 2 | Epoch 005 | Loss: 0.6878 | Train Acc: 0.5350 | Test Acc: 0.5800
Seed 2 | Epoch 006 | Loss: 0.6764 | Train Acc: 0.5813 | Test Acc: 0.5300
Seed 2 | Epoch 007 | Loss: 0.6855 | Train Acc: 0.5288 | Test Acc: 0.5650
Seed 2 | Epoch 008 | Loss: 0.6806 | Train Acc: 0.5613 | Test Acc: 0.5300
Seed 2 | Epoch 009 | Loss: 0.6783 | Train Acc: 0.5513 | Test Acc: 0.5750
Seed 2 | Epoch 010 | Loss: 0.6735 | Train Acc: 0.5988 | Test Acc: 0.5700
Seed 2 | Epoch 011 | Loss: 0.6671 | Train Acc: 0.5975 | Test Acc: 0.5600
Seed 2 | Epoch 012 | Loss: 0.6665 | Train Acc: 0.5875 | Test Acc: 0.6000
Seed 2 | Epoch 013 | Loss: 0.6597 | Train Acc: 0.6100 | Test Acc: 0.5950
Seed 2 | Epoch 014 | Loss: 0.6578 | Train Acc: 0.63

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,███▇▇▇▇▇▆▆▅▅▅▅▅▄▄▅▄▄▃▄▃▃▃▃▃▃▂▃▂▂▃▂▁▂▁▁▁▁
test_acc,▁▅▁▅▂▂▅▅▄▆▂▆▅▆▆▆▄▆▄▆▅▅▅▄▅▅▅▅▅▅▄▅▄▆▆▇▅▅█▅
train_acc,▁▂▂▃▂▃▃▄▄▄▆▅▆▆▆▆▇▆▆▆▆▆▆▇▇▆▆▇▇▇▇▇▆▇▇▇████
epoch,50
loss,0.58917
test_acc,0.575
train_acc,0.69875


...

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇█
loss,███▇▇▆▇▆▆▆▆▅▅▅▅▄▅▄▄▄▄▃▄▃▃▃▃▃▃▂▂▂▁▂▂▁▁▁▁▁
test_acc,▂▃▅▁▁▅▆▆▇▇▄▇▇▆▇▆▇▇▆▇▆▆▇▇▅▆▇▆▆▆▇▆▇▄█▆▆█▆▅
train_acc,▁▂▂▃▂▃▃▄▅▅▅▅▅▅▅▆▆▇▆▅▆▆▇▆▆▇▆▇▆▅▇█▇▇▇▇▇█▇▇
epoch,50
loss,0.62365
test_acc,0.59
train_acc,0.645


Seed 29 | Epoch 001 | Loss: 0.6945 | Train Acc: 0.5088 | Test Acc: 0.4850
Seed 29 | Epoch 002 | Loss: 0.6894 | Train Acc: 0.5288 | Test Acc: 0.4850
Seed 29 | Epoch 003 | Loss: 0.6887 | Train Acc: 0.5425 | Test Acc: 0.5400
Seed 29 | Epoch 004 | Loss: 0.6840 | Train Acc: 0.5425 | Test Acc: 0.4850
Seed 29 | Epoch 005 | Loss: 0.6815 | Train Acc: 0.5613 | Test Acc: 0.5600
Seed 29 | Epoch 006 | Loss: 0.6780 | Train Acc: 0.5837 | Test Acc: 0.5850
Seed 29 | Epoch 007 | Loss: 0.6730 | Train Acc: 0.5938 | Test Acc: 0.6200
Seed 29 | Epoch 008 | Loss: 0.6693 | Train Acc: 0.5825 | Test Acc: 0.5800
Seed 29 | Epoch 009 | Loss: 0.6705 | Train Acc: 0.5875 | Test Acc: 0.5900
Seed 29 | Epoch 010 | Loss: 0.6696 | Train Acc: 0.5837 | Test Acc: 0.5850
Seed 29 | Epoch 011 | Loss: 0.6662 | Train Acc: 0.6200 | Test Acc: 0.6000
Seed 29 | Epoch 012 | Loss: 0.6644 | Train Acc: 0.6075 | Test Acc: 0.5800
Seed 29 | Epoch 013 | Loss: 0.6638 | Train Acc: 0.5975 | Test Acc: 0.5700
Seed 29 | Epoch 014 | Loss: 0.6548 | T

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
loss,███▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▅▅▃▅▄▄▃▃▃▃▄▃▃▃▃▃▂▂▂▂▁▂
test_acc,▁▃▁▄▅▅▅▅▆▅▅▅▅▇▅▆▆▅▇▅▆▄▅▆▆▆▇▄▆▇▆▅▆▇█▄▆▅▇▅
train_acc,▁▂▂▂▃▄▄▄▄▅▄▅▆▅▆▅▆▆▅▆▆▆▅▆▆▇▇▇▇▇▇▆▇▇▇▇▇██▇
epoch,50
loss,0.61935
test_acc,0.59
train_acc,0.66


Seed 30 | Epoch 001 | Loss: 0.6981 | Train Acc: 0.5062 | Test Acc: 0.5650
Seed 30 | Epoch 002 | Loss: 0.6933 | Train Acc: 0.5100 | Test Acc: 0.4550
Seed 30 | Epoch 003 | Loss: 0.6903 | Train Acc: 0.5312 | Test Acc: 0.4350
Seed 30 | Epoch 004 | Loss: 0.6890 | Train Acc: 0.5275 | Test Acc: 0.4550
Seed 30 | Epoch 005 | Loss: 0.6828 | Train Acc: 0.5487 | Test Acc: 0.6000
Seed 30 | Epoch 006 | Loss: 0.6815 | Train Acc: 0.5837 | Test Acc: 0.6100
Seed 30 | Epoch 007 | Loss: 0.6730 | Train Acc: 0.6062 | Test Acc: 0.5600
Seed 30 | Epoch 008 | Loss: 0.6697 | Train Acc: 0.6088 | Test Acc: 0.5650
Seed 30 | Epoch 009 | Loss: 0.6666 | Train Acc: 0.5913 | Test Acc: 0.6200
Seed 30 | Epoch 010 | Loss: 0.6665 | Train Acc: 0.5938 | Test Acc: 0.5050
Seed 30 | Epoch 011 | Loss: 0.6662 | Train Acc: 0.5988 | Test Acc: 0.5850
Seed 30 | Epoch 012 | Loss: 0.6569 | Train Acc: 0.6225 | Test Acc: 0.6000
Seed 30 | Epoch 013 | Loss: 0.6593 | Train Acc: 0.6100 | Test Acc: 0.5400
Seed 30 | Epoch 014 | Loss: 0.6573 | T

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,██▇▇▇▆▆▆▆▆▅▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▂▁▁
test_acc,▆▂▁▂▇▆▆█▄▇▄▇▅██▆▆▆▃▄▆▆▇▇▇▇▇▇█▇▇▆▇▇▇▇▇▇▇▆
train_acc,▁▁▂▂▃▅▅▅▅▆▆▆▆▆▅▆▆▇▇▆▆▇▇█▇▇▇▇▇▇█▇▇▇██████
epoch,50
loss,0.60118
test_acc,0.575
train_acc,0.66875


# **BFS and centrality**

In [4]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch import nn
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = f"GCN Graph classification({Dataset_name})_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, cent_dim=16):
        super().__init__()
        self.cent_enc = nn.Linear(1, cent_dim)
        self.conv1 = GCNConv(in_channels + cent_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, out_channels)
    def forward(self, x, cent2, edge_index, batch):
        if x is None:
            x = torch.ones((cent2.shape[0], 1), device=cent2.device)
        x = torch.cat([x, self.cent_enc(cent2.unsqueeze(1))], dim=1)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def make_centrality_transform():
    def transform(data):
        G = to_networkx(data, to_undirected=True)
        n = data.num_nodes
        deg_cent = nx.degree_centrality(G) if G.number_of_nodes() > 0 else {}
        cent = torch.zeros((n,), dtype=torch.float)
        for u in range(n):
            cent[u] = float(deg_cent.get(u, 0.0))
        data.cent2 = cent
        if getattr(data, "x", None) is None:
            data.x = torch.ones((n, 1), dtype=torch.float)
        return data
    return transform

def run_seed(SEED):
    set_seed(SEED)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dataset = TUDataset(root="data/TUDataset", name=Dataset_name, transform=make_centrality_transform())
    torch.manual_seed(SEED)
    dataset = dataset.shuffle()
    num_graphs = len(dataset)
    train_size = int(0.8 * num_graphs)
    train_dataset = dataset[:train_size]
    test_dataset = dataset[train_size:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_CentralityOnly_seed_{SEED}",
        group=f"GCN_{Dataset_name}_CentralityOnly",
        job_type="centrality-only",
        reinit=True,
        config={
            "model": "GCN",
            "hidden_channels": 64,
            "epochs": EPOCHS,
            "lr": 0.001,
            "dataset": Dataset_name,
            "task": "Centrality_Only",
            "seed": SEED,
            "cuda_available": torch.cuda.is_available(),
            "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
            "cuda_version": torch.version.cuda if torch.cuda.is_available() else None
        }
    )

    model = GCN(
        in_channels=dataset.num_features if dataset.num_features else 1,
        hidden_channels=64,
        out_channels=dataset.num_classes,
        cent_dim=16
    ).to(device)

    opt = torch.optim.Adam(model.parameters(), lr=0.001)

    def train():
        model.train()
        total_loss = 0.0
        correct = 0
        total = 0
        for data in train_loader:
            data = data.to(device)
            out = model(data.x, data.cent2, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            opt.zero_grad()
            loss.backward()
            opt.step()
            total_loss += loss.item() * data.num_graphs
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            total += data.num_graphs
        return total_loss / len(train_dataset), correct / total

    def test():
        model.eval()
        correct = 0
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.cent2, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
        return correct / len(test_dataset)

    for epoch in range(1, EPOCHS + 1):
        loss, train_acc = train()
        test_acc = test()
        print(f"Seed {SEED} | Epoch {epoch:03d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})

    wandb.finish()

if __name__ == "__main__":
    wandb.login()
    for s in range(1, NUM_SEEDS + 1):
        run_seed(s)

Seed 1 | Epoch 001 | Loss: 0.6957 | Train Acc: 0.5375 | Test Acc: 0.6250
Seed 1 | Epoch 002 | Loss: 0.6871 | Train Acc: 0.5337 | Test Acc: 0.6200
Seed 1 | Epoch 003 | Loss: 0.6874 | Train Acc: 0.5475 | Test Acc: 0.6100
Seed 1 | Epoch 004 | Loss: 0.6837 | Train Acc: 0.5487 | Test Acc: 0.6150
Seed 1 | Epoch 005 | Loss: 0.6859 | Train Acc: 0.5675 | Test Acc: 0.6300
Seed 1 | Epoch 006 | Loss: 0.6830 | Train Acc: 0.5863 | Test Acc: 0.6300
Seed 1 | Epoch 007 | Loss: 0.6846 | Train Acc: 0.5775 | Test Acc: 0.6000
Seed 1 | Epoch 008 | Loss: 0.6839 | Train Acc: 0.5887 | Test Acc: 0.6200
Seed 1 | Epoch 009 | Loss: 0.6826 | Train Acc: 0.5825 | Test Acc: 0.6250
Seed 1 | Epoch 010 | Loss: 0.6848 | Train Acc: 0.5800 | Test Acc: 0.6150
Seed 1 | Epoch 011 | Loss: 0.6883 | Train Acc: 0.5575 | Test Acc: 0.6200
Seed 1 | Epoch 012 | Loss: 0.6825 | Train Acc: 0.5675 | Test Acc: 0.6150
Seed 1 | Epoch 013 | Loss: 0.6816 | Train Acc: 0.5750 | Test Acc: 0.6300
Seed 1 | Epoch 014 | Loss: 0.6841 | Train Acc: 0.55

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇███
loss,█▄▄▃▄▃▃▂▃▅▂▃▃▃▂▂▃▃▃▃▂▂▂▂▂▂▂▂▂▃▂▂▁▁▃▂▃▁▂▁
test_acc,▆▅▁▃█▅▆▃▅▃▁▆▅▅▆▃▅▅▆▃▅▆▅▆▅▅▆▅▃▅▅▆▆▅▅▅▅▅▅▅
train_acc,▁▁▃▃▅▆█▇▇▄▆▃▆▆▆▄▅▅▇▆▇▇██▇▇▆▇▇█▇▇▆▅▆▇▇▆▇▇
epoch,50
loss,0.67941
test_acc,0.62
train_acc,0.58375


Seed 2 | Epoch 001 | Loss: 0.6948 | Train Acc: 0.5050 | Test Acc: 0.4750
Seed 2 | Epoch 002 | Loss: 0.6861 | Train Acc: 0.5513 | Test Acc: 0.5750
Seed 2 | Epoch 003 | Loss: 0.6823 | Train Acc: 0.5750 | Test Acc: 0.5450
Seed 2 | Epoch 004 | Loss: 0.6804 | Train Acc: 0.5825 | Test Acc: 0.5750
Seed 2 | Epoch 005 | Loss: 0.6840 | Train Acc: 0.5737 | Test Acc: 0.5700
Seed 2 | Epoch 006 | Loss: 0.6798 | Train Acc: 0.5938 | Test Acc: 0.5550
Seed 2 | Epoch 007 | Loss: 0.6812 | Train Acc: 0.5887 | Test Acc: 0.5600
Seed 2 | Epoch 008 | Loss: 0.6786 | Train Acc: 0.5988 | Test Acc: 0.5750
Seed 2 | Epoch 009 | Loss: 0.6788 | Train Acc: 0.5962 | Test Acc: 0.5750
Seed 2 | Epoch 010 | Loss: 0.6804 | Train Acc: 0.5913 | Test Acc: 0.5550
Seed 2 | Epoch 011 | Loss: 0.6794 | Train Acc: 0.5900 | Test Acc: 0.5750
Seed 2 | Epoch 012 | Loss: 0.6790 | Train Acc: 0.5988 | Test Acc: 0.5750
Seed 2 | Epoch 013 | Loss: 0.6797 | Train Acc: 0.5813 | Test Acc: 0.5600
Seed 2 | Epoch 014 | Loss: 0.6769 | Train Acc: 0.59

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▅▃▃▄▃▂▂▃▂▂▁▂▂▂▂▂▂▂▃▂▂▁▂▂▁▁▁▁▂▂▁▁▂▂▁▁▁▂▁
test_acc,▁▆█▇▆██▆███▆███▇███▃▇▇█████████████▇█▇██
train_acc,▁▄▆▇▆▇██▇▇▇█▇▇███▇██▅█▇███▇▇███▇▇▇▇▇██▆▇
epoch,50
loss,0.67593
test_acc,0.58
train_acc,0.58875


Seed 3 | Epoch 001 | Loss: 0.6906 | Train Acc: 0.4938 | Test Acc: 0.5550
Seed 3 | Epoch 002 | Loss: 0.6843 | Train Acc: 0.5837 | Test Acc: 0.5500
Seed 3 | Epoch 003 | Loss: 0.6795 | Train Acc: 0.5863 | Test Acc: 0.5700
Seed 3 | Epoch 004 | Loss: 0.6815 | Train Acc: 0.5637 | Test Acc: 0.5900
Seed 3 | Epoch 005 | Loss: 0.6793 | Train Acc: 0.5837 | Test Acc: 0.5200
Seed 3 | Epoch 006 | Loss: 0.6788 | Train Acc: 0.5587 | Test Acc: 0.5850
Seed 3 | Epoch 007 | Loss: 0.6819 | Train Acc: 0.5713 | Test Acc: 0.5950
Seed 3 | Epoch 008 | Loss: 0.6784 | Train Acc: 0.5863 | Test Acc: 0.5500
Seed 3 | Epoch 009 | Loss: 0.6793 | Train Acc: 0.5750 | Test Acc: 0.5950
Seed 3 | Epoch 010 | Loss: 0.6814 | Train Acc: 0.5737 | Test Acc: 0.5850
Seed 3 | Epoch 011 | Loss: 0.6793 | Train Acc: 0.5825 | Test Acc: 0.5450
Seed 3 | Epoch 012 | Loss: 0.6792 | Train Acc: 0.5825 | Test Acc: 0.6000
Seed 3 | Epoch 013 | Loss: 0.6798 | Train Acc: 0.5663 | Test Acc: 0.5650
Seed 3 | Epoch 014 | Loss: 0.6776 | Train Acc: 0.58

...

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█▄▇▄▃▄▃▄▃▃▃▃▂▂▃▂▃▂▂▂▂▂▂▁▂▂▃▂▂▂▃▁▁▂▁▂▁▂▁▁
test_acc,▇▁▅█▂▆▅▅▆▅▆▅▅▆▆▆▅▅▆▅▅▆▅▆▅▅▆▆▅▆▆▅▅▆▅▅▆▅▇▆
train_acc,▁▆▄▅▆▆▇▇▇▇▇▇▇▇▇▇▇█▇▇▇█▆▇████▇▇█▆██▇▇█▇▇█
epoch,50
loss,0.67432
test_acc,0.555
train_acc,0.6025


Seed 29 | Epoch 001 | Loss: 0.6965 | Train Acc: 0.5188 | Test Acc: 0.5500
Seed 29 | Epoch 002 | Loss: 0.6889 | Train Acc: 0.5312 | Test Acc: 0.6200
Seed 29 | Epoch 003 | Loss: 0.6834 | Train Acc: 0.5837 | Test Acc: 0.5750
Seed 29 | Epoch 004 | Loss: 0.6847 | Train Acc: 0.5663 | Test Acc: 0.5250
Seed 29 | Epoch 005 | Loss: 0.6835 | Train Acc: 0.5725 | Test Acc: 0.6200
Seed 29 | Epoch 006 | Loss: 0.6820 | Train Acc: 0.5800 | Test Acc: 0.6100
Seed 29 | Epoch 007 | Loss: 0.6811 | Train Acc: 0.5925 | Test Acc: 0.6200
Seed 29 | Epoch 008 | Loss: 0.6810 | Train Acc: 0.5850 | Test Acc: 0.6200
Seed 29 | Epoch 009 | Loss: 0.6803 | Train Acc: 0.5837 | Test Acc: 0.6050
Seed 29 | Epoch 010 | Loss: 0.6841 | Train Acc: 0.5787 | Test Acc: 0.5400
Seed 29 | Epoch 011 | Loss: 0.6805 | Train Acc: 0.5800 | Test Acc: 0.6150
Seed 29 | Epoch 012 | Loss: 0.6820 | Train Acc: 0.5875 | Test Acc: 0.6100
Seed 29 | Epoch 013 | Loss: 0.6819 | Train Acc: 0.5700 | Test Acc: 0.5450
Seed 29 | Epoch 014 | Loss: 0.6811 | T

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▅▃▃▃▂▂▂▃▂▂▂▃▂▁▂▂▂▁▁▁▂▁▁▁▂▂▂▂▁▂▁▂▁▂▃▂▂▁▂
test_acc,▂█▄█▇█▇▁█▇█▇█▃████▇▇▂█▇███▇██▇▇▇▇▇▇▄█▇▇▇
train_acc,▁▂▇▆▆█▇▇▇█▇▄▇▆▇▇▇▆▇▇▇▇▇▇▇▇█▇▇▆█▆▇▆▆▇█▇▇▆
epoch,50
loss,0.68008
test_acc,0.61
train_acc,0.57125


Seed 30 | Epoch 001 | Loss: 0.6950 | Train Acc: 0.5038 | Test Acc: 0.4350
Seed 30 | Epoch 002 | Loss: 0.6900 | Train Acc: 0.5150 | Test Acc: 0.5900
Seed 30 | Epoch 003 | Loss: 0.6882 | Train Acc: 0.5463 | Test Acc: 0.6000
Seed 30 | Epoch 004 | Loss: 0.6826 | Train Acc: 0.5663 | Test Acc: 0.5250
Seed 30 | Epoch 005 | Loss: 0.6829 | Train Acc: 0.5850 | Test Acc: 0.5900
Seed 30 | Epoch 006 | Loss: 0.6867 | Train Acc: 0.5450 | Test Acc: 0.5950
Seed 30 | Epoch 007 | Loss: 0.6821 | Train Acc: 0.5863 | Test Acc: 0.6050
Seed 30 | Epoch 008 | Loss: 0.6841 | Train Acc: 0.5800 | Test Acc: 0.6000
Seed 30 | Epoch 009 | Loss: 0.6838 | Train Acc: 0.5775 | Test Acc: 0.5850
Seed 30 | Epoch 010 | Loss: 0.6852 | Train Acc: 0.5525 | Test Acc: 0.6050
Seed 30 | Epoch 011 | Loss: 0.6846 | Train Acc: 0.5487 | Test Acc: 0.6400
Seed 30 | Epoch 012 | Loss: 0.6822 | Train Acc: 0.5787 | Test Acc: 0.6000
Seed 30 | Epoch 013 | Loss: 0.6813 | Train Acc: 0.5863 | Test Acc: 0.6000
Seed 30 | Epoch 014 | Loss: 0.6821 | T

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▆▅▂▂▃▃▃▂▁▄▂▂▂▂▂▁▂▂▁▁▂▁▁▁▂▁▁▂▂▁▁▁▁▁▂▁▁▂▁
test_acc,▁▆▇▄▆▇▇▆▇█▇▇▄▇▆▆▇▇▇▇▆▇▆▇▆▇▇▇▆▆▆▇▇▇▇▇▆▇▇▇
train_acc,▁▂▄▆▇▇▇▇▅▄▇▇▄█▆▇▆▇█▇▇█▇██▆▇▇█▇▇▆▆█▇███▇▆
epoch,50
loss,0.68133
test_acc,0.6
train_acc,0.5625
